# Moodify

Song Recommendation based on Users’ Emotion Recognition System.

In [22]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from google.colab import files, drive
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt

In [20]:
drive.mount('/content/gdrive')

train_dir = "/content/gdrive/MyDrive/train"
test_dir  = "/content/gdrive/MyDrive/test"

assert os.path.isdir(train_dir), "train_dir not found – check the path/ls output"
assert os.path.isdir(test_dir),  "test_dir not found – check the path/ls output"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.9/217.9 kB 5.9 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 18.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.6/29.6 MB 27.9 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.1.3
    Uninstalling numpy-2.1.3:
      Successfully uninstalled numpy-2.1.3
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


# Set up

In [ ]:
size = 48
batch_size = 64

train_data_generator = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.1
)

train_set = train_data_generator.flow_from_directory(
    train_dir,
    target_size=(size, size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_set = train_data_generator.flow_from_directory(
    train_dir,
    target_size=(size, size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

test_data_generator = ImageDataGenerator(
    rescale=1./255
)

test_set = test_data_generator.flow_from_directory(
    test_dir,
    target_size=(size, size),
    color_mode='grayscale',
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
def create_model_predict_emotion(num_classes=7):
    input_shape = (48, 48, 1)
    model = tf.keras.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(96, (3,3), activation='relu', padding='same'),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    model.compile(
        optimizer=tf.keras.optimizers.Adam(1e-3),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
def plot_model(history):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['accuracy'], label='train_acc')
    plt.plot(history.history['val_accuracy'], label='val_acc')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.title('Train vs Val Accuracy')
    plt.subplot(1,2,2)
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('Train vs Val Loss')
    plt.tight_layout()
    plt.show()

In [ ]:
def accuracy_on_test_set(model):
    test_loss, test_acc = model.evaluate(test_set)
    print(f"Test accuracy: {test_acc:.2f}")
    print(f"Test loss: {test_loss:.2f}")

In [ ]:
def fit_and_save_model(path="best_model.h5"):
    model = create_model_predict_emotion()
    model.summary()   
    epochs = 1
    checkpoint_path = path
    steps_per_epoch = min(len(train_set), 100)

    callbacks = [
        EarlyStopping(
            monitor='val_accuracy',
            patience=4,
            restore_best_weights=True
        ),
        ModelCheckpoint(
            checkpoint_path,
            monitor='val_accuracy',
            save_best_only=True,
            verbose=1
        )
    ]

    history = model.fit(
        train_set,
        validation_data=val_set,
        epochs=epochs,
        callbacks=callbacks,
        steps_per_epoch=steps_per_epoch
    )
    plot_model(history)


fit_and_save_model()

In [ ]:
emotion_labels = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]

def process_img(img_path):
    img = load_img(img_path, color_mode='grayscale', target_size=(size, size))
    x = img_to_array(img).astype("float32") / 255.0
    x = np.expand_dims(x, axis=0) 
    return x

def upload_predict_and_save(model, mood_txt_path="current_mood.txt"):
    print("Upload an image of your face.")
    uploaded = files.upload()
    if not uploaded:
        print("No file uploaded.")
        return

    img_name = list(uploaded.keys())[0]
    probs = model.predict(process_img(img_name))
    emotion = emotion_labels[int(np.argmax(probs))]

    with open(mood_txt_path, "w") as f:
        f.write(emotion + "\n")

    print(f"Predicted emotion: {emotion}")

In [ ]:
model = load_model("best_model.h5") 
upload_predict_and_save(model, mood_txt_path="current_mood.txt")